# QC Report 

This interactive notebook generates a summary of QC statistics to assess the success of a **Mad4hatter** run. 

## Required Inputs 

To proceed, you must provide the **results directory** from the Mad4hatter pipeline, which should include the following files:

* **sample_coverage.txt**
* **amplicon_coverage.txt**
* **allele_data.txt**

Additionally, a **sample manifest** is required. This file must contain the following fields:

* **SampleID** – Unique identifier for each sample.
* **SampleType** – Specifies whether the entry is a **sample**, **positive** control, or **negative** control.
* **Batch** – Identifies a group of samples processed simultaneously by the same individual.
* **Column** – The well column where the sample was placed in the plate.
* **Row** – The well row where the sample was placed in the plate.
* **Parasitemia** – The qPCR value for the sample.

## QC Summaries
The notebook provides the following analyses:

* **Plate Layout**: Location of samples and controls on the plate
* **Primer Dimer Content**: Input reads vs. % attributed to primer dimers 
* **Balancing Across Batches**: Swarm plot of reads output by the pipeline by batch
* **Control Summary**, including:  
    * Polyclonality of Positive Controls
    * Read Summary for Negative Controls
    * Contamination Maps for Negative Controls
* **Successul Amplification** summary, including:
    * Sample Reads vs. Number of Successfully Amplified Loci
    * Parasitemia vs. Number of Successfully Amplified Loci
* **Amplification Plate Maps**, including:
    * Reads Heatmap
    * Successful Amplification Heatmap
 
      
Note: A locus is considered successfully amplified if it has more than `read_threshold` reads, where `read_threshold` is a threshold that can be set below.

## Setup

In [ ]:
# User input 
# Input path to results directory from Mad4hatter pipeline
results_dir <- "PATH/TO/MAD4HATTER_RESULTS_DIR/"
# Input path to manifest 
manifest_file <- "PATH/TO/MANIFEST"
# Path to outputs
reprep_repool_summary_path <- "reprep_repool_summary.csv"
polyclonal_information_path <- "positive_control_polyclonal_info.csv"
neg_control_information_path <- "negative_control_amplified_loci.csv"
filtered_allele_data_path <- "allele_data_filtered.txt"

In [ ]:
# If TRUE, trims extra sequencing-related information from SampleID to match the manifest format
standardise_sampleID <- TRUE
# Input a threshold for the reads needed to classify a locus as having amplified successfully
read_threshold <- 100
# Input a minimum Reads per ASV used as a filter for the positive control plot
read_filter <- 0 
# Minimum Allele Frequency per ASV used as a filter for positive control plot 
af_filter <- 0.01 
# Read threshold to output locus information for negative control
negative_control_read_threshold <- 50 
# Proportion of targets successfully amplified; samples below this threshold require re-prep
reprep_threshold <- 0.5 
# Proportion of targets successfully amplified; samples below this threshold require re-pool
repool_threshold <- 0.75
# This column indicates the one to be used for an allele ID, either ASV or PseudoCIGAR. 
# If PseudoCIGAR, the table will be collapsed to unique PseudoCIGARs.
allele_col <- "PseudoCIGAR"

In [ ]:
# Print summary of user inputs
cat("User Input Summary\n")
cat("Results Directory:", results_dir, "\n")
cat("Manifest File:", manifest_file, "\n")
cat("Read Threshold for Successful Amplification:", read_threshold, "\n")
cat("Minimum Reads per ASV (Positive Control Filter):", read_filter, "\n")
cat("Minimum Allele Frequency per ASV (Positive Control Filter):", af_filter, "\n")

In [ ]:
# Create file paths 
sample_coverage_file <- file.path(results_dir,"sample_coverage.txt")
amplicon_coverage_file <- file.path(results_dir,"amplicon_coverage.txt")
allele_data_path <- file.path(results_dir, "allele_data.txt")

In [ ]:
# Load necessary libraries
library(ggplot2)
library(plotly, quietly = TRUE,warn.conflicts = FALSE)
library(dplyr, quietly = TRUE,warn.conflicts = FALSE)
library(tidyr)
library(ggbeeswarm)
library(stringr)
library(tibble)
library(kableExtra, quietly = TRUE, warn.conflicts = FALSE)

In [ ]:
manifest <- read.csv(manifest_file) %>% mutate(Row = toupper(Row)) # Ensure Row is capitalised
sample_coverage <- read.csv(sample_coverage_file, sep="\t")
allele_data <- read.csv(allele_data_path, sep='\t')

In [ ]:
# Load coverage per amplicon and add on amplicon length
amplicon_coverage <- read.delim(amplicon_coverage_file)

In [ ]:
amplicon_coverage <- amplicon_coverage %>%
  mutate(SampleID = if (standardise_sampleID) SampleID %>%
           sub("_L001$", "", .) %>%
           word(., start = 1, end = -2, sep = "_") else SampleID)
allele_data <- allele_data %>%
  mutate(SampleID = if (standardise_sampleID) SampleID %>%
           sub("_L001$", "", .) %>%
           word(., start = 1, end = -2, sep = "_") else SampleID)

Here we filter out two types of loci:

* **Loci targeting non-Plasmodium falciparum species**, which are expected to amplify only if other species are present.

* **Long amplicons (>275 bp, including primers)**, which tend to underperform due to their length.

These loci should not be considered when assessing the success of a sequencing run.

**Note**: If you modify the filtered loci, be sure to update the loci counts below accordingly.

In [ ]:
# Species Loci
species_loci <- c('Pf3D7_13_v3-1041593-1041860-1AB',
                'PmUG01_12_v1-1397996-1398245-1AB',
                'PocGH01_12_v1-1106456-1106697-1AB',
                'PvP01_12_v1-1184983-1185208-1AB',
                'PKNH_12_v2-198869-199113-1AB')
# Long loci 
long_loci <- c('Pf3D7_14_v3-854128-854427-1B',
                'Pf3D7_11_v3-1294284-1294580-1B',
                'Pf3D7_13_v3-1465010-1465307-1B',
                'Pf3D7_13_v3-2841390-2841688-1B2',
                'Pf3D7_13_v3-2844334-2844629-1B2',
                'Pf3D7_02_v3-320652-320949-2',
                'Pf3D7_03_v3-240957-241256-2',
                'Pf3D7_05_v3-615379-615678-2',
                'Pf3D7_06_v3-857454-857753-2')

# Combine into a list of all excluded loci
excluded_loci <- c(species_loci, long_loci)

# Filter amplicon_coverage to exclude rows with loci in the exclusion list
amplicon_coverage <- amplicon_coverage[!amplicon_coverage$Locus %in% excluded_loci, ]

# Filter allele_data to exclude rows with loci in the exclusion list
allele_data <- allele_data[!allele_data$Locus %in% excluded_loci, ]

Below are the loci counts per reaction, set with the expectation that you are running **Madh4hatter pools D1.1, R1.2, and R2.1.**. The loci we filtered out above are excluded from these counts. 

* If you are **not** using these pools, please update these numbers manually.
* Alternatively, you can uncomment the section below to calculate loci counts directly from the `allele_table`. **Note:** If a locus fails to amplify in all samples, it will not be included in the counts when derived from the `allele_table`.

In [ ]:
# D1.1 has 170 loci in total. 5 are removed above
pool_d1.1_loci <- 165
# R1.2 has 48 loci in total. 8 are removed above
pool_r1.2_loci <- 40
# R2.1 has 31 loci in total. 6 are removed above
pool_r2.1_loci <- 25

# Loci per reaction  
reaction1_loci <- pool_d1.1_loci + pool_r1.2_loci
reaction2_loci <- pool_r2.1_loci

# Put into a table
nloci_table <- data.frame(
  reaction = c('1', '2'),
  nreactionloci = c(reaction1_loci, reaction2_loci)
)
nloci_table

Uncomment the following to count the number of loci per reaction based on the `allele_data` table.

In [ ]:
# nloci_table <- allele_data %>%
#   distinct(Locus) %>% 
#   mutate(reaction = str_extract(Locus, "\\d(?=[A-Z]*$)")) %>%
#   count(reaction) %>%
#   rename(nreactionloci = n) 
# nloci_table

In [ ]:
merge_data <- function(manifest, sample_coverage, standardise_sampleID) {
    merged_data <- sample_coverage %>%
        mutate(SampleID = if (standardise_sampleID) SampleID %>%
               sub("_L001$", "", .) %>%
               word(., start = 1, end = -2, sep = "_") else SampleID) %>%
        mutate(Reads = as.numeric(Reads)) %>% 
        replace_na(list(Reads = 0)) %>%
        pivot_wider(names_from = Stage, values_from = Reads) %>%
        inner_join(manifest, by = "SampleID") 
    return(merged_data)
}

In [ ]:
# Reformat sample_coverage and add sample information from manifest 
sample_coverage_with_manifest <- merge_data(manifest, sample_coverage, standardise_sampleID)

In [ ]:
# Add manifest to amplicon coverage
amplicon_coverage_with_manifest <- amplicon_coverage %>%
    inner_join(manifest,by = c("SampleID"))

In [ ]:
# Add reaction to amplicon table 
# reaction here refers to 1 of the 2 mPCR reactions reaction1 : 1A+5, reaction2 : 2.
# Note: in future versions of the pipeline this may have to come from the panel information, not the locus name 
amplicon_coverage_with_manifest <- amplicon_coverage_with_manifest %>%
  mutate(reaction = str_extract(Locus, "\\d(?=[A-Z]*$)"))

In [ ]:
# Create sample summary stats 
summary_samples <- amplicon_coverage_with_manifest %>%
  # Join the nloci_table
  left_join(nloci_table, by = "reaction") %>%
  # Calculate number of loci with reads over threshold per reaction
  group_by(SampleID, Batch, SampleType, reaction, nreactionloci) %>%
  summarize(
    reads_per_reaction = sum(OutputPostprocessing),
    n_good_loci = sum(OutputPostprocessing > read_threshold),
    .groups = "drop"
  ) %>%
    group_by(SampleID, Batch, SampleType) %>%
      mutate(
        reads_per_sample = sum(reads_per_reaction),
      ) %>%
  ungroup() %>%
  mutate(prop_good_loci = n_good_loci / nreactionloci) %>%
  inner_join(manifest, by = join_by(SampleID, Batch, SampleType))

In [ ]:
# Plotting Setup
sample_colours <- c(
  "negative" = "red3",
  "positive" = "blue3",
  "sample" = "darkgrey",
  "empty" = "black"
)

In [ ]:
reaction_colours <- c("skyblue", "orangered2", "turqoise3")

In [ ]:
# Print summary of samples in files
cat("Sample Summary\n")
cat(length(unique(manifest$SampleID)), "samples in manifest", "\n")
cat(length(unique(allele_data$SampleID)), "samples in allele data file", "\n")
cat(length(unique(sample_coverage$SampleID)), "samples in manifest and sample coverage file", "\n")
cat(length(unique(amplicon_coverage$SampleID)), "samples in manifest and amplicon coverage file", "\n")

## Plate Layout

The plate layout provides a visual representation of sample organization, including the placement of positive and negative controls. This is allows verification that controls are positioned correctly to validate assay performance with later plots.

In [ ]:
create_plate_template <- function(summary_samples, batches, nrows=8, ncols=12) {
    quadrants <-
      expand.grid(
        Batch = batches,
        y = 1:nrows,
        Column = 1:ncols
      ) %>%
      mutate(
        ymin = y - 0.45,
        ymax = y + 0.45,
        xmin = Column - 0.45,
        xmax = Column + 0.45
      ) %>%
      mutate(Row = toupper(rev(letters[1:nrows])[y])) %>%
      # Join on the sample information to the plate
      left_join(
        summary_samples %>%
          select(Batch, Column, Row, SampleType) %>%
          distinct(),
        by = c("Batch", "Column", "Row" = "Row")
      )
    return(quadrants)
}

In [ ]:
quadrants <- create_plate_template(summary_samples, unique(summary_samples$Batch))

In [ ]:
plot_plate_layout <- function(quadrants, sample_colours) {
    # Ensure that NAs in SampleType are replaced with "NA" string (or another label)
    quadrants$SampleType[is.na(quadrants$SampleType)] <- "empty"
    plate_layout <- ggplot(quadrants) +
      # Tiles representing sample types, fill based on SampleType
      geom_tile(aes(x = Column, y = y, fill = SampleType), color = "white") +
      # Facet by Batch
      facet_wrap(vars(Batch), ncol = 1) +
      scale_y_continuous(breaks = quadrants$y, labels = quadrants$Row) +
      # Make sure labels are correct
      xlab("Column") +
      ylab("Row") +
      # Customizing the x-axis
      scale_x_continuous(breaks = 1:12, labels = as.character(1:12)) +
      # Quadrants as rectangles with borders
      geom_rect(
        data = quadrants,
        aes(xmin = xmin, xmax = xmax, ymin = ymin, ymax = ymax),
        color = "black", fill = NA, linewidth = 1
      ) +
      # Custom fill colors for each SampleType
      scale_fill_manual(
        values = sample_colours
      ) +
      # Theming
      theme_minimal() +
      theme(
        axis.text = element_text(size = 12),  
        strip.text = element_text(size = 14, face = "bold"), 
        panel.grid = element_blank(),  
        plot.title = element_text(size = 16, face = "bold", hjust = 0.5),
      ) +
    # Customizing the legend to add an outline for the colored squares
    guides(
      fill = guide_legend(
        override.aes = list(
          color = "black",  # Outline color for the squares
          size = 1          # Outline size for the squares
        )
      )
    ) +
    # Title for the plot
    ggtitle("Sample Types and Quadrants Distribution by Batch")+
    coord_fixed(ratio = 0.75)
    print(plate_layout)
}

In [ ]:
plot_plate_layout(quadrants, sample_colours)

## Primer Dimer Content

Here we visualise the proportion of sequencing reads that are classified as primer dimers, which occur when primers anneal to each other instead of the target DNA. This plot is useful for assessing the efficiency of the amplification process, as high primer dimer levels can indicate suboptimal reaction conditions, reduced sequencing efficiency, and potential issues with sample quality or reagent performance.

In [ ]:
# Function to generate the plot
generate_dimer_plot <- function(sample_coverage_with_manifest, sample_colours) {
    
    dimer_plot <- ggplot(data = sample_coverage_with_manifest %>%  arrange(desc(SampleType))) +
      # Plot points with different colors for each sample type
      geom_point(aes(x = Input + 0.9, 
                     y = (1 - OutputPostprocessing / Input) * 100, 
                     color = SampleType),
                 shape = 1, 
                 alpha=0.8,
                 stroke = 1) +
    # Log scale for the x-axis
    scale_x_log10() +
    # Faceting by Batch column
    facet_wrap(~Batch) +
    # Adding labels and title
    ylab("% Dimers") +
    xlab("Input Reads") +
    ggtitle("Dimer Content")+
    # Color scale for SampleType
    scale_color_manual(values = sample_colours) 
    print(dimer_plot)
}

In [ ]:
generate_dimer_plot(sample_coverage_with_manifest, sample_colours)

## Balancing Across Batches

Here we show the distribution of total reads per sample across different batches, helping to assess whether sequencing depth is consistent. This plot is useful when you have multiple batches in the same sequencing run to identify imbalances in sequencing, which can arise due to variations in library preparation, loading efficiency, or sequencing conditions.

In [ ]:
generate_balancing_plot <- function(summary_samples, sample_colours) {
  ggplot() + 
    ggbeeswarm::geom_quasirandom(
    data = summary_samples %>% 
        select(SampleID, Batch, reads_per_sample, SampleType) %>% 
        distinct(), 
    aes(x = Batch, y = reads_per_sample + 0.9, color = SampleType)
    ) +
    scale_y_log10() +
    scale_color_manual(values = sample_colours) +
    ylab("Total Reads for Sample") +
    ggtitle("Balancing Across Batches") +
    theme(
      axis.text.x = element_text(angle = 90, hjust = 1)  # Rotate x-axis labels by 45 degrees
    )
}

In [ ]:
generate_balancing_plot(summary_samples, sample_colours)

## Control Summary

### Polyclonality of Positive Controls

Here we inspect the positive controls, ensuring that the positive controls perform as expected without contamination or unwanted diversity. For example, if the control you included was monoclonal you would expect little to no loci reported as having more than one allele (polyclonal). In some cases you may allow for some level of false positive detection within a monoclonal control, these filters 

In [ ]:
# Add on within sample allele freqeucny 
allele_data <- allele_data %>%
  group_by(SampleID, Locus) %>%
  mutate(AlleleFreq = Reads / sum(Reads)) %>%
  ungroup()

In [ ]:
pos_control_allele_data <- allele_data %>%
  inner_join(manifest %>% filter(SampleType == "positive"), by = "SampleID")

In [ ]:
if (allele_col=="PseudoCIGAR") {
  pos_control_data <- pos_control_allele_data %>%
      group_by(SampleID, Locus, PseudoCIGAR) %>%
      summarise(Reads = sum(Reads), AlleleFreq = sum(AlleleFreq), .groups = "drop")
} else {
  pos_control_data <- pos_control_allele_data
}

In [ ]:
# Load required libraries
# Apply filters and identify the number of alleles being called per locus
locus_summary <- pos_control_data %>%
  group_by(SampleID, Locus) %>%
  summarise(
    NumASVs_Meeting_Threshold = sum(Reads > read_filter & AlleleFreq > af_filter),
    .groups = "drop"
  ) %>%
  mutate(
    Category = case_when(
      NumASVs_Meeting_Threshold == 1 ~ "Monoclonal",
      NumASVs_Meeting_Threshold == 2 ~ "2 Alleles",
      NumASVs_Meeting_Threshold > 2  ~ ">2 Alleles"
    )
  )

# Count loci per sample for each category
result <- locus_summary %>%
  group_by(SampleID, Category) %>%
  summarise(Count = n(), .groups = "drop")

# Create ggplot
p <- ggplot(result, aes(x = SampleID, y = Count, fill = Category, text = paste("Count:", Count))) +
  geom_bar(stat = "identity", position = "stack") +
  labs(title = "Clonality of Loci for Positive Controls", x = "Sample ID", y = "Number of Loci") +
  theme_minimal() +
  scale_fill_manual(values = c("Monoclonal" = "Gray", "2 Alleles" = "orangered2", ">2 Alleles" = "turquoise3")) + 
  theme(
    axis.text.x = element_text(angle = 90, hjust = 1)
  )

# Convert ggplot to interactive plot
ggplotly(p, tooltip = "text", width = 800, height = 500)

In [ ]:
# Filter data to keep only loci with more than one allele (polyclonal cases)
polyclonal_information <- pos_control_data %>%
  inner_join(locus_summary, by = c("SampleID", "Locus")) %>%
  filter(Category %in% c("2 Alleles", ">2 Alleles"))  # Keep only polyclonal categories

# Save to CSV
write.csv(polyclonal_information, polyclonal_information_path, row.names = FALSE)

# Display as styled table
head(polyclonal_information) |>
  kable() |>
  kable_styling(bootstrap_options = c("striped", "hover", "condensed"))

### Negative Control Contamination 

#### Read Summary per Negative Control
In an ideal scenario, negative controls should have minimal or no amplification. If a negative control shows a high number of targets amplified with significant reads, it suggests potential contamination. By plotting the number of reads against the number of targets for each sample, any outliers or unexpected amplification in negative controls can be easily flagged.

In [ ]:
# retrieve negative controls from amplicons.full
amplicons_negative = amplicon_coverage_with_manifest %>% 
  filter(SampleType=="negative") %>% 
  mutate(negative = paste0(Batch," Well: ",toupper(Row),Column)) 

In [ ]:
ggplot(amplicons_negative) +
  geom_histogram(aes(x = OutputPostprocessing, fill = reaction)) +
  facet_wrap(~ negative) +
  xlim(-ifelse(
    max(amplicons_negative$OutputPostprocessing) == 0,
    1e2,
    max(amplicons_negative$OutputPostprocessing)
  ) / 50,
  ifelse(
    max(amplicons_negative$OutputPostprocessing) == 0,
    1e2,
    max(amplicons_negative$OutputPostprocessing) + 10 
  )) +
  xlab("Reads") +
  ylab("Count") + 
  scale_fill_manual(values = reaction_colours)

In [ ]:
neg_control_information <- amplicons_negative %>%
    filter(OutputPostprocessing > negative_control_read_threshold) %>%
    select('SampleID','Locus','OutputPostprocessing')
write.csv(neg_control_information, neg_control_information_path, row.names = FALSE)
head(neg_control_information_path) |>
  kable() |>
  kable_styling(bootstrap_options = c("striped", "hover", "condensed"))

#### Aggregated Reads per Target

This plot aggregates the total reads for each locus across all negative controls. In negative controls, if there is an unexpected spike in reads for specific loci, it could indicate contamination in the form of cross-sample contamination or environmental contamination. Analyzing these total summed reads helps pinpoint specific loci where contamination might have occurred, offering insights into which steps in the process might have introduced contaminants.

In [ ]:
# Get reads for each locus, label reaction and extract chrom and start from locus name
summary_data <- amplicons_negative %>%
  group_by(reaction, Locus) %>%
  summarise(sum_reads = sum(Reads, na.rm = TRUE), .groups = "drop") %>%
  mutate(
    Chromosome = sub("-.*", "", Locus),  
    Start = as.numeric(sub(".*-(\\d+)-.*", "\\1", Locus))
  ) %>%
  arrange(Chromosome, Start) %>%
  mutate(Locus = factor(Locus, levels = unique(Locus))) %>%  
  rowid_to_column("RowID") 

# Add a column with a list of sample IDs where Reads > 0 for each locus
summary_data <- amplicons_negative %>%
  filter(Reads > 0) %>%
  group_by(Locus) %>%
  summarise(samples_with_reads = paste(unique(SampleID), collapse = ", "), .groups = "drop") %>%
  right_join(summary_data, by = "Locus")

# Reduce x-axis labels: label every nth locus
nth_label <- 20 
summary_data <- summary_data %>%
  mutate(Locus_label = ifelse(RowID %% nth_label == 1, paste0(Chromosome, ":", Start), ""))

# Create the bar plot with hover text
p <- ggplot(summary_data, aes(x = Locus, y = sum_reads, fill = reaction, text = paste("Locus: ", Locus, "<br>Total Reads: ", sum_reads, "<br>Controls with Reads: ", samples_with_reads))) +
  geom_bar(stat = "identity", position = "dodge") +
  xlab("Locus (Chromosome:Start Coordinate)") +
  ylab("Sum of Reads Across Controls") +
  ggtitle("Total Reads per Locus Across Negative Controls") +
  theme(
    axis.text.x = element_text(angle = 90, hjust = 1, size = 8),
    axis.title = element_text(size = 12),
    panel.grid = element_blank(),
    panel.background = element_blank()
  ) +
  scale_x_discrete(labels = summary_data$Locus_label) +  # Apply reduced labels
  scale_fill_manual(values = reaction_colours)

# Convert to interactive plotly plot
ggplotly(p, tooltip = "text")

#### Negative Control Contamination Maps

In [ ]:
plot_plate_with_feature <- function(target, allele_data, quadrants, sample_colours, fill_param, scale_midpoint, scale_label) {

    target_data <- allele_data %>% filter(SampleID == target)
    comparison <- target_data %>% 
        inner_join(allele_data, by = c("Locus","PseudoCIGAR"), suffix = c("_target", "")) %>%
        group_by(SampleID_target, SampleID, PseudoCIGAR) %>%
        summarise(
            common_count = n(),  # Count the number of times the PseudoCIGAR is shared
            common_reads = sum(Reads_target + Reads, na.rm = TRUE),  # Sum the reads of both samples
            .groups = "drop"
        )
    comparison_plate <- comparison %>%
      inner_join(manifest, by = "SampleID")
    
    # Merge comparison_plate with quadrants to include reads_per_reaction
    merged_data <- merge(comparison_plate, quadrants, by = c("Batch", "Row", "Column", "SampleType"), all.y = TRUE)
    
    # Ensure NAs in SampleType are labeled
    merged_data$SampleType[is.na(merged_data$SampleType)] <- "empty"
    
    # Create a new column to identify the target sample
    merged_data$target <- ifelse(merged_data$SampleID == target, "target", "other")

    # Create the heatmap plot
    plate_layout <- ggplot(merged_data) +
      # Heatmap fill based on reads_per_reaction
      geom_tile(aes(x = Column, y = y, fill = !!rlang::parse_expr(fill_param)), color = NA, width = 0.95, height = 0.95) +
      # Separate tile layer for outlines
      geom_rect(data = merged_data,
                aes(xmin = Column-0.45, xmax = Column+0.45, ymin = y-0.45, ymax = y+0.45, color = target), 
                fill = NA, linewidth = 1.3) +
      # Color gradient for heatmap
      scale_fill_gradient2(
        low = "black",
        mid = "darkorange4",
        high = "darkorange",
        midpoint = scale_midpoint,
        name = scale_label
      ) +
      # Custom colors for target vs normal
      scale_color_manual(values = c("other" = "darkgrey", "target" = "red3")) +  # Highlight sample in red, others in gray
      # Labels
      scale_y_continuous(breaks = merged_data$y, labels = merged_data$Row) +
      scale_x_continuous(breaks = 1:12, labels = as.character(1:12)) +
      xlab("Column") +
      ylab("Row") +
      ggtitle("Reads per Reaction Heatmap") +
      # Theme adjustments
      theme_minimal() +
      theme(
        axis.text = element_text(size = 12),
        strip.text = element_text(size = 14, face = "bold"),
        panel.grid = element_blank(),
        plot.title = element_text(size = 16, face = "bold", hjust = 0.5),
        legend.key = element_rect(colour = "black")  # Ensure visible legend border
      ) + 
      coord_fixed(ratio = 0.75)
    
    print(plate_layout)
}

In [ ]:
plot_plate_with_feature("Negative-Control-2", allele_data, quadrants, sample_colours, "log10(common_reads+0.1)", 2, "log10(common_reads)")

## Successul Amplification

### Parasitemia vs. Number of Successfully Amplified Loci
This plot examines the relationship between a sample's parasitemia (qPCR-determined parasite load) and the number of loci that successfully amplified. This plot helps assess whether lower parasitemia samples struggle with amplification, which can indicate potential limitations in sensitivity.

In [ ]:
generate_parasitemia_by_amplification_success_plot <- function(summary_samples, threshold, sample_colours) {
    parasitemia_plot <- ggplot(data = summary_samples) +
      # Plot points with different colors for each sample type
      geom_point(aes(x = Parasitemia + 0.9, 
                     y = prop_good_loci, 
                     color = SampleType),
                 shape = 1,
                 alpha=0.8,
                 stroke = 1) +
    # Log scale for the x-axis
    scale_x_log10() +
    ylim(0,1) + 
    # Faceting by Batch column
    facet_grid(cols = vars(Batch),rows = vars(reaction),
               scale = "free_y")+
    # Adding labels and title
    ylab(paste0("Amplicons with >",threshold," reads")) +
    xlab("Parasitemia (log10)") +
    theme(axis.text.x = element_text(angle = 90, hjust = 1))+

    ggtitle("Amplicons with `good` read depth")+
    # Color scale for SampleType
    scale_color_manual(values = sample_colours) 
    print(parasitemia_plot)
}

In [ ]:
generate_parasitemia_by_amplification_success_plot(summary_samples, read_threshold, sample_colours)

### Sample Reads vs. Number of Successfully Amplified Loci

This illustrates the relationship between the total number of reads per sample and the number of loci that passed the amplification threshold for that sample. This plot helps evaluate whether samples with higher read counts achieve better amplification success and can reveal potential issues such as insufficient sequencing depth or inefficient amplification. Ideally, a positive correlation should be observed, where higher read counts result in more successfully amplified loci.

In [ ]:
generate_reads_by_amplification_success_plot <- function(summary_samples, amplification_threshold, 
                                                         sample_colours, reprep_threshold, repool_threshold) {
    
    amp_plot <- ggplot(data = summary_samples) +
      # Plot points with different colors for each sample type
      geom_point(aes(x = reads_per_sample + 0.9,  
                     y=prop_good_loci,
                     color = SampleType),
                 shape=1,
                 alpha=0.8,
                 stroke = 1) +
    # Log scale for the x-axis
    scale_x_log10() +
    # Fix y-axis scale from 0 to 1
    scale_y_continuous(limits = c(0, 1)) +
    # Faceting by Batch column
    facet_grid(cols = vars(Batch),rows = vars(reaction),
               scale = "free_y")+
    # Adding labels and title
    ylab(paste0("Amplicons with >",amplification_threshold," reads")) +
    xlab("Total Reads for Sample") +
    theme(axis.text.x = element_text(angle = 90, hjust = 1))+

    ggtitle("Targets that Amplified Successfully")+
    # Color scale for SampleType
    scale_color_manual(values = sample_colours) +
    # Add dashed horizontal lines at reprep_threshold and repool_threshold
    geom_hline(yintercept = reprep_threshold, linetype = "dashed") +
    geom_hline(yintercept = repool_threshold, linetype = "dashed") +
    # Add labels for the horizontal lines
    annotate("text", x = min(summary_samples$reads_per_sample, na.rm = TRUE), 
               y = reprep_threshold, label = "Reprep", vjust = 1, hjust = 0) +
    annotate("text", x = min(summary_samples$reads_per_sample, na.rm = TRUE), 
               y = repool_threshold, label = "Repool", vjust = 1, hjust = 0) 
    print(amp_plot)
}

In [ ]:
generate_reads_by_amplification_success_plot(summary_samples, read_threshold, sample_colours,  reprep_threshold, repool_threshold)

In [ ]:
generate_reprep_repool_table <- function(summary_samples, reprep_threshold, repool_threshold, output_path) {
    summary_table <- summary_samples %>%
      mutate(
          status = case_when(
          prop_good_loci < reprep_threshold ~ "reprep",
          prop_good_loci < repool_threshold ~ "repool",
          TRUE ~ "pass"
        ),
        reason = case_when(
          prop_good_loci < reprep_threshold ~ paste("< reprep threshold ",reprep_threshold),
          prop_good_loci < repool_threshold ~ paste("< repool threshold ",repool_threshold),
          TRUE ~ "NA"
        )
      ) %>%
      arrange(desc(status)) %>%
      select(SampleID, Batch, reaction, status, reason, reads_per_reaction)
    
    return(summary_table)
}

In [ ]:
# Add in any samples that were in manifest and not in results
fill_missing_data <- function(reprep_repool_summary, manifest) {
  # Get unique reactions
  unique_reactions <- unique(reprep_repool_summary$reaction)

  # Identify missing samples from manifest
  missing_samples <- manifest %>%
    filter(!(SampleID %in% reprep_repool_summary$SampleID)) %>%
    select(SampleID, Batch)

  # Create rows for missing SampleIDs with all reactions
  missing_sample_rows <- missing_samples %>%
    crossing(reaction = unique_reactions) %>%
    mutate(
      status = "reprep",
      reason = "no data for sample",
      reads_per_reaction = 0
    )
  
  # Ensure all existing samples have all reactions
  missing_reaction_rows <- reprep_repool_summary %>%
    select(SampleID, Batch) %>%
    distinct() %>%
    crossing(reaction = unique_reactions) %>%
    anti_join(reprep_repool_summary, by = c("SampleID", "Batch", "reaction")) %>%
    mutate(
      status = "reprep",
      reason = "no data for reaction",
      reads_per_reaction = 0
    )

  # Combine everything
  bind_rows(reprep_repool_summary, missing_sample_rows, missing_reaction_rows)
}

In [ ]:
reprep_repool_summary <- generate_reprep_repool_table(summary_samples, reprep_threshold, repool_threshold, reprep_repool_summary_path)
reprep_repool_summary <- fill_missing_data(reprep_repool_summary, manifest)
head(reprep_repool_summary) |>
  kable() |>
  kable_styling(bootstrap_options = c("striped", "hover", "condensed"))

In [ ]:
write.csv(reprep_repool_summary, reprep_repool_summary_path, row.names = FALSE)

## Amplification Plate Maps

### Reads Heatmap

The reads heatmap provides a visual representation of read distribution across the plate, helping to identify inconsistencies in sequencing efficiency. This can highlight potential issues such as edge effects, batch effects, or pipetting errors that may impact data quality and interpretation.

In [ ]:
plot_plate_with_feature <- function(summary_samples, quadrants, sample_colours, fill_param, scale_midpoint, scale_label) {
    # Merge summary_samples with quadrants to include reads_per_reaction
    quadrants <- crossing(summary_samples %>% select(reaction) %>% distinct() , quadrants)
    merged_data <- merge(summary_samples, quadrants, by = c("Batch", "Row", "Column", "SampleType", "reaction"), all.y = TRUE)
    
    # Ensure NAs in SampleType are labeled
    merged_data$SampleType[is.na(merged_data$SampleType)] <- "empty"
    
    # Create the heatmap plot
    plate_layout <- ggplot(merged_data) +
      # Heatmap fill based on reads_per_reaction
      geom_tile(aes(x = Column, y = y, fill = !!rlang::parse_expr(fill_param)), color = NA, width = 0.95, height = 0.95) +
      # Separate tile layer for outlines
      geom_rect(data = merged_data,
            aes(xmin = Column-0.45, xmax = Column+0.45, ymin = y-0.45, ymax = y+0.45,color=SampleType),fill = NA,linewidth=1.3)+
      # Facet by Batch and reaction
      facet_grid(reaction ~ Batch) +
      # Color gradient for heatmap
      scale_fill_gradient2(
        low = "black",
        mid = "darkorange4",
        high = "darkorange",
        midpoint = scale_midpoint,
        name = scale_label
      ) +
      # Outline colors based on SampleType
      scale_color_manual(values = sample_colours, na.translate = TRUE) +
      # Labels
      scale_y_continuous(breaks = merged_data$y, labels = merged_data$Row) +
      scale_x_continuous(breaks = 1:12, labels = as.character(1:12)) +
      xlab("Column") +
      ylab("Row") +
      ggtitle("Reads per Reaction Heatmap") +
      # Theme adjustments
      theme_minimal() +
      theme(
        axis.text = element_text(size = 12),
        strip.text = element_text(size = 14, face = "bold"),
        panel.grid = element_blank(),
        plot.title = element_text(size = 16, face = "bold", hjust = 0.5),
        legend.key = element_rect(colour = "black")  # Ensure visible legend border
      ) + 
    coord_fixed(ratio = 0.75)
    print(plate_layout)
}

In [ ]:
plot_plate_with_feature(summary_samples, quadrants, sample_colours, "log10(reads_per_reaction + 0.1)", 2, "log10(reads_per_reaction)")

### Successful Amplification Heatmap
Here we visualise the success rate of amplification across the plate, allowing for the identification of poorly amplified regions or wells. This is useful for assessing the quality of the PCR process, ensuring that loci across all samples are adequately amplified, and helping to spot potential issues with specific wells or sample groups.

In [ ]:
plot_plate_with_feature(summary_samples, quadrants, sample_colours, "prop_good_loci", 0.5, "prop_good_loci")

## Save filtered allele table

In [ ]:
# Read in raw data with no other filtering applied 
allele_data <- read.csv(allele_data_path, sep='\t')
# If using pseudocigar column, then group by this for reads
if (allele_col=="PseudoCIGAR") {
  allele_data <- allele_data %>%
      group_by(SampleID, Locus, PseudoCIGAR) %>%
      summarise(Reads = sum(Reads), .groups = "drop")
}
# Calculate allele frequency
allele_data <- allele_data %>%
  group_by(SampleID, Locus) %>%
  mutate(AlleleFreq = Reads / sum(Reads)) %>%
  ungroup()
# Filter to minimum read and within sample allele frequency 
allele_data_filtered <- allele_data %>% 
  filter(Reads > read_filter & AlleleFreq > af_filter)
# Write out table 
write.csv(allele_data_filtered, filtered_allele_data_path, row.names = FALSE)